In [1]:
%matplotlib inline

The project is a simulation of a rocket that is launched from Earth and aims to land on Mars.

In [2]:
import math

from time import sleep
from random import sample
from os import remove

from collections import namedtuple, deque
from itertools import count

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from skimage.io import imread, imshow
from skimage.transform import resize

import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

from vpython import *

<IPython.core.display.Javascript object>

# Landing Rocket On Mars

### UPDATE

What's new?
1. I take the semi-axes of the ellipses of the two planets and after each restart of the environment they start from a random place. (Here we use the ellipse equation).
2. Calculate the velocity vector for the position of the planet after the reset of the environment.
3. I use the law of conservation of kinetic energy and the law of conservation of momentum to change the basic velocity of the rocket.
4. added fuel to the rocket, which will affect the reward for each step.
5. I reduced the number of network parameters.

This makes the problem more complicated.

Let me describe how I calculate the planet's velocity vector at a given point after a reboot. We use Kepler's second law. We know from it that for equal intervals of time a planet travels the same area speed. Let's define some constants that we will use to solve the problem:

$\sigma$ - area velocity<br>
$r_{x}$ - x coordinate of the radius vector<br>
$r_{y}$ - y coordinate of the radius vector<br>
$a$ - major semi-axis<br>
$b$ - minor semi-axis<br>
$\theta$ - angle between the two vectors

We know how much $\sigma$ is because we have a basic case in which we know the speed of the planet, the radius-vector, and the angle between the two vectors.

$\vec\sigma = \vec v \vec r \sin(\theta)$ = det $\begin{pmatrix} r_{x} & r_{y} \\ v_{x} & v_{y} \end{pmatrix}$

det $\begin{pmatrix} r_{x} & r_{y} \\ v_{x} & v_{y} \end{pmatrix}$ = $r_{x}v_{y} - r_{y}v_{x}$ = $|\vec\sigma|$

$\Rightarrow v_{y}(v_{x}) = \frac{\sigma + r_{y}v_{x}}{r_{x}}$

Тhe two vectors form a parallelogram, I express the small diagonal of the parallelogram in two ways.

$d_{2}^2 = (r_{x} - v_{x})^2 + (r_{y} - v_{y})^2 = (r_{x} - v_{x})^2 + (r_{y} - \frac{\sigma + r_{y}v_{x}}{r_{x}})^2$<br><br>
$d_{2}^2 = \vec r^2 + \vec v^2 - 2|\vec r||\vec v|\cos(\theta) = \vec r^2 + v_{x}^2 + (\frac{\sigma + r_{y}v_{x}}{r_{x}})^2 - 2|\vec r|\sqrt {v_{x}^2 + (\frac{\sigma + r_{y}v_{x}}{r_{x}})^2}\cos(\theta)$<br><br>
$\Rightarrow \frac{r_{y}}{r_{x}}(\sigma+r_{y}v_{x}) + r_{x}v{x} = |\vec r|\sqrt {v_{x}^2 + (\frac{\sigma + r_{y}v_{x}}{r_{x}})^2}\cos(\theta)$<br><br>
We know that $|\vec\sigma| = |\vec r|\sqrt {v_{x}^2 + (\frac{\sigma + r_{y}v_{x}}{r_{x}})^2}\sin(\theta)$<br><br>
$\Rightarrow \tan(\theta)(\frac{r_{y}}{r_{x}}(\sigma + r_{y}v_{x}) + r_{x}v_{x}) = \sigma$

The line on which the velocity vector lies is tangent to the ellipse.<br>
Let $y = k_{1}x + c_{1}$ is the equation of the line of velocity vector.<br>
$\Rightarrow a^2k_{1}^2 + b^2 = c_{1}^2, c_{1} = \sqrt{a^2k_{1}^2 + b^2}$<br>
After replacement we get:<br>
$k_{1} = \frac{r_{x}r_{y}}{r_{x}^2-a^2}$<br>
Let $y = k_{2}x + c_{2}$ is the equation of line of radius-vector.<br>
Since the line passes through the center of the coordinate system, this means that $c_{2} = 0$ and $k_{2} = \frac{r_{y}}{r_{x}}$

$\tan(\theta) = \frac{k_{2} - k_{1}}{1 + k_{1}k_{2}} = a^2r_{y}(a^2-r^2)$

$\tan(\theta)(\frac{r_{y}}{r_{x}}(\sigma + r_{y}v_{x}) + r_{x}v_{x}) = \sigma$<br>
$a^2r_{y}(a^2-r^2)(\frac{r_{y}}{r_{x}}(\sigma + r_{y}v_{x}) + r_{x}v_{x}) = \sigma$<br><br>
$\Rightarrow v_{x} = \frac{\sigma r_{x}}{a^2 r^2 r_{y}(a^2 - r^2)} - \frac{r_{y}}{r^2}\sigma$

$\vec v = (\frac{\sigma r_{x}}{a^2 r^2 r_{y}(a^2 - r^2)} - \frac{r_{y}}{r^2}\sigma, \frac{\sigma + r_{y}v_{x}}{r_{x}}, 0)$

Both momentum and kinetic energy are conserved, so for bodies with masses $m_{1}$and $m_{2}$, and velocities $u_{1}$ and $u_{2}$ before collision, and $v_{1}$ and $v_{2}$ after collision. The momentum before and after the collision is expressed by:

$m_{1}u_{1} + m_{2}u_{2} = m_{1}v_{1} + m_{2}v_{2}$

The kinetic energy is expressed by:<br>
$\frac{m_{1}u_{1}}{2}+\frac{m_{2}u_{2}}{2} = \frac{m_{1}v_{1}}{2} + \frac{m_{2}v_{2}}{2}$

The solution of the system is:<br>
$v_{1} = \frac{m_{1}-m_{2}}{m_{1}+m_{2}}u_{1} + \frac{2m_{2}}{m_{1}+m_{2}}u_{2}$<br>
$v_{2} = \frac{2m_{1}}{m_{1}+m_{2}}u_{1} + \frac{m_{2}-m_{1}}{m_{1}+m_{2}}u_{2}$<br>

Since $m_{1} \ll m_{2}$:<br>
$ v_{1} \approx -u_{1} + 2u_{2} $<br>
$ v_{2} \approx u_{2} $

At the beginnig let us define the constants we will use.

It is important to specify that the parameters of the planets and the sun will not be real, because the simulation will not work well enough to show the result.

In [3]:
grav_const = 6.67e-11
dt = 1

EARTH_RADIUS = 20
MARS_RADIUS = 17
SUN_RADIUS = 55

max_number_of_steps = 50

gamma = 0.99
batch_size = 18
replay_size = 100
learning_rate = 0.8
epsilon = 1

MEAN_REWARD_BOUND = 30

eps_start = 0.5
eps_decay = .99
eps_min = 0.02

fuel = 100

sigma_earth = mag(vector(200, 0, 0)) * mag(vector(0, -8, 0))
sigma_mars = mag(vector(282, 0, 0)) * mag(vector(0, -8 * 0.8, 0))

def vx(pos, a, b, sigma):
    return (sigma * pos.x) / (a*a*mag(pos)**2*pos.y*(a*a-mag(pos)**2)) - pos.y / mag(pos)**2 * sigma

def vy(pos, sigma, x):
    return (sigma + pos.y * x) / pos.x

In nutshell, the constants are as follows:<br><hr>
`grav_const` - the gravitational constant of the law of universal gravitation<br>
`dt` - the change in time by one step<br><hr>
`EARTH_RADIUS` - radius of Earth<br>
`MARS_RADIUS` - radius of Mars<br>
`SUN_RADIUS` - radius of the sun<br><hr>
`gamma` - the constant we will use when changing q-values<br>
`batch_size` - the amount of information we need to have to start training the network<br>
`replay_size` - the maximum amount of stored information<br>
`learning_rate` - learning rate<br>
`epsilon` - coefficient for a random event<br><hr>
`eps_start` - the initial value of `epsilon`<br>
`eps_decay` - the rate of change of the random event<br>
`eps_min` - minimum value of the random event<br><hr>
`MEAN_REWARD_BOUND` - the minimum reward for completing the network training

Let's create the model architecture.

In [4]:
model = Sequential([
    Input((400, 640, 4)),
    Conv2D(filters = 64, kernel_size = 3, padding = 'valid', activation = 'relu'),
    MaxPool2D(),
    Conv2D(filters = 32, kernel_size = 3, padding = 'valid', activation = 'relu'),
    MaxPool2D(),
    BatchNormalization(),
    Conv2D(filters = 16, kernel_size = 3, padding = 'valid', activation = 'relu'),
    MaxPool2D(),
    Conv2D(filters = 8, kernel_size = 3, padding = 'valid', activation = 'relu'),
    MaxPool2D(),
    Flatten(),
    Dense(64, activation = 'relu'),
    Dropout(0.1),
    Dense(32, activation = 'relu'),
    Dropout(0.1),
    Dense(16, activation = 'relu'),
    Dense(8, activation = 'sigmoid')
])

The model consists of 3 convolutional layers, MaxPool for image size reduction, BatchNormalization for scaling and 3 hidden layers. The reason we have 8 output neurons is that we have 8 possible directions for movement in the environment.

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 398, 638, 64)      2368      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 199, 319, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 197, 317, 32)      18464     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 98, 158, 32)       0         
_________________________________________________________________
batch_normalization (BatchNo (None, 98, 158, 32)       128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 96, 156, 16)       4624      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 48, 78, 16)        0

In [6]:
model.compile(optimizer = Adam(learning_rate), loss = 'mse')

We create two classes, using the classes from `vpython` to make our work with data more convenient.

We have two methods in the `Planet` class that serve to change the position of the planets and change the speed of the planet.

In [7]:
class Star:
    def __init__(self, mass, radius, pos, color):
        self.star = sphere(pos = pos, radius = radius, color = color)
        self.mass = mass

class Planet:
    def __init__(self, mass, radius, pos, velocity, color):
        self.planet = sphere(pos = pos, radius = radius, color = color, make_trail = False)
        self.mass = mass
        if radius == EARTH_RADIUS:
            self.planet.texture = textures.earth
        self.planet.v = velocity
        self.momentum = mag(velocity) * mass
        
    def update_params(self, Fgrav):
        # self.momentum += Fgrav * dt
        # print('Velocity: ', self.planet.v, 'Pos: ', self.planet.pos)
        self.planet.v += Fgrav * dt # / self.mass
    
    def move(self):
        self.planet.pos += self.planet.v * dt

Using the code below, we create our environment, and then we create a `reset_env()` function so we can restart everything.

`total_reward` is the reward for the current performance, and `total_steps` is the number of steps we have taken since the rocket took off.

`dist_from_sun` is a dictionary that stores information about the distances from the sun to Earth, Mars and the rocket.

In [8]:
total_reward = 0.0
total_steps = 0

celestial_bodies = {
    'Sun': Star(1e10/2, SUN_RADIUS, vector(0, 0, 0), color.orange),
    'Earth': Planet(1e5/3, EARTH_RADIUS, vector(200, 0, 0), vector(0, -8, 0), color.blue),
    'Mars': Planet(1e4/3, MARS_RADIUS, vector(282, 0, 0), vector(0, -8*0.8, 0), color.red)
}
    
dist_from_sun = {}
    
rocket = cylinder(
    pos = vector(200, EARTH_RADIUS, 0), 
    axis = vector(0, 3, 0), 
    length = 11,
    velocity = vector(0, 0, 0),
    radius = 3.4)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
major_axis_earth = 0
minor_axis_earth = 0

major_axis_mars = 0
minor_axis_mars = 0

def collected_info(a1, b1, a2, b2):
    if a1 != 0 and b1 != 0 and a2 != 0 and b2 != 0:
        return True
    return False

In [11]:
while True:
    simulation()
    
    if collected_info(major_axis_earth, minor_axis_earth, major_axis_mars, minor_axis_mars):
        break
    
    if -5 < celestial_bodies['Earth'].planet.pos.x < 5:
        minor_axis_earth = abs(celestial_bodies['Earth'].planet.pos.y)
    if -5 < celestial_bodies['Earth'].planet.pos.y < 5 and celestial_bodies['Earth'].planet.pos.x < 0:
        major_axis_earth = (200 + abs(celestial_bodies['Earth'].planet.pos.x)) / 2
    
    if -5 < celestial_bodies['Mars'].planet.pos.x < 5:
        minor_axis_mars = abs(celestial_bodies['Mars'].planet.pos.y)
    if -5 < celestial_bodies['Mars'].planet.pos.y < 5 and celestial_bodies['Mars'].planet.pos.x < 0:
        major_axis_mars = (282 + abs(celestial_bodies['Mars'].planet.pos.x)) / 2

In [12]:
(major_axis_earth, minor_axis_earth), (major_axis_mars, minor_axis_mars)

((235.8012320191254, 226.33431532390125),
 (293.45783811303306, 289.7154496903096))

In [13]:
y_earth = lambda x: minor_axis_earth / major_axis_earth * math.sqrt((200 - x) * (2 * major_axis_earth + x - 200))
y_mars = lambda x: minor_axis_mars / major_axis_mars * math.sqrt((282 - x) * (2 * major_axis_mars + x - 282))

Unlike the above code, here we also have a return to the initial values of some of the variables.

In [14]:
def reset_env():
    
    global total_reward, total_steps, epsilon, frame, eps_start, fuel
    
    eps_start *= eps_decay
    epsilon = eps_start / 2
    total_reward = 0.0
    total_steps = 0
    frame = 0
    fuel = 100
    
    global celestial_bodies
    for celestial_body in celestial_bodies.values():
        try:
            celestial_body.planet.visible = False
        except:
            celestial_body.star.visible = False
    try:        
        x_start_earth = np.random.randint(200 - 2 * major_axis_earth, 200)
        print('x_start_earth: ', x_start_earth)
        x_start_mars = np.random.randint(282 - 2 * major_axis_mars, 282)
    except:
        reset_env()
        return
    
    celestial_bodies = {
        'Sun': Star(1e10/2, SUN_RADIUS, vector(0, 0, 0), color.orange),
        'Earth': Planet(1e5/3, EARTH_RADIUS, vector(x_start_earth, y_earth(x_start_earth), 0), vector(0, -8, 0), color.blue),
        'Mars': Planet(1e4/3, MARS_RADIUS, vector(x_start_mars, y_mars(x_start_mars), 0), vector(0, -8*0.8, 0), color.red)
    }
    
    celestial_bodies['Earth'].planet.v = vector(vx(celestial_bodies['Earth'].planet.pos, major_axis_earth, minor_axis_earth, sigma_earth), vy(celestial_bodies['Earth'].planet.pos, sigma_earth, vx(celestial_bodies['Earth'].planet.pos, major_axis_earth, minor_axis_earth, sigma_earth)), 0)
    celestial_bodies['Mars'].planet.v = vector(vx(celestial_bodies['Mars'].planet.pos, major_axis_mars, minor_axis_mars, sigma_mars), vy(celestial_bodies['Mars'].planet.pos, sigma_mars, vx(celestial_bodies['Mars'].planet.pos, major_axis_mars, minor_axis_mars, sigma_mars)), 0)
    
    global dist_from_sun
    dist_from_sun = {}
    
    global rocket
    rocket.visible = False
    rocket = cylinder(
        pos = vector(x_start_earth, y_earth(x_start_earth) + EARTH_RADIUS, 0), 
        axis = vector(0, 3, 0), 
        length = 11, 
        radius = 3.4)

In [15]:
reset_env()

x_start_earth:  149


`Experience` is the class through which we will keep information about the `reward` we have received after performing an `action` from position `state` to position `next_state`.

In [16]:
Experience = namedtuple(
    'Experience',
    ('state', 'action', 'next_state', 'reward', 'done')
)

`ReplayMemory` is the class in which we will store the objects of the `Experience` class, as the maximum stored information is 100.

In [17]:
class ReplayMemory:
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = deque(maxlen = capacity)
        
    def push(self, experience):
        self.memory.append(experience)
        
    def sample(self, batch_size):
        indices = np.random.choice(len(self.memory), batch_size, replace = False)
        states, actions, next_states, rewars, dones = zip([self.memory[ind] for ind in indices])
        return np.array(states), np.array(actions), np.array(next_states), np.array(rewards, dtype=np.float32), np.array(dones, dtype=np.uint8)

The `push` function adds an object to the `memory` list, and if it is full, it starts to fill from the beginning.

The `sample()` function takes n number of random examples from `memory`, where n = `batch_size`.

With the code below we create the class `Agent`, which has the functions for taking action - `play_step()`, performing action - `step()` and the function for training our network - `train()`.

In [21]:
class Agent:
    
    def __init__(self, exp_buffer):
        self.exp_buffer = exp_buffer
        
    def calculate_dist_to_sun(position):
        return mag(rocket.pos)
    
    def landing():
        dist = mag(rocket.pos - celestial_bodies['Mars'].planet.pos)
        
        if dist <= celestial_bodies['Mars'].planet.radius:
            return True
        return False
    
    def dist_mars_rocket():
        return mag(rocket.pos - celestial_bodies['Mars'].planet.pos)
    
    def landed_on_earth():
        dist = ((celestial_bodies['Earth'].planet.pos.x - rocket.pos.x)**2 +
               (celestial_bodies['Earth'].planet.pos.y - rocket.pos.y)**2 +
               (celestial_bodies['Earth'].planet.pos.z - rocket.pos.z)**2)**0.5
        
        if dist <= celestial_bodies['Earth'].planet.radius:
            return True
        return False


    def train(self, net, target_net):
        batch = sample(self.exp_buffer.memory, batch_size)
        
        for state, action, next_state, reward, done in batch:
            
            target = net.predict(np.array([state]))
            
            if done:
                target[0][action] = reward
            else:
                t = target_net.predict(np.array([next_state]))
                target[0][action] = reward + gamma * np.amax(t)
                
            net.fit(np.array([state]), np.array(target), epochs=1, verbose=0)
        
    def step(action):
        
        global frame_idx, total_steps, frame, fuel
        is_done = False
        
        d_dist_before_action = Agent.dist_mars_rocket()
        
        rocketVelocity = vector(0, 0, 0)
        
        if action == 0:
            rocketVelocity.x -= 10
            rocketVelocity.y += 10
        elif action == 1:
            rocketVelocity.y += 10
        elif action == 2:
            rocketVelocity.x += 10
            rocketVelocity.y += 10
        elif action == 3:
            rocketVelocity.x -= 10
        elif action == 4:
            rocketVelocity.x += 10
        elif action == 5:
            rocketVelocity.x -= 10
            rocketVelocity.y -= 10
        elif action == 6:
            rocketVelocity.y -= 10
        elif action == 7:
            rocketVelocity.x += 10
            rocketVelocity.y -= 10
            
        rocketVelocity = -rocketVelocity + 2 * celestial_bodies['Earth'].planet.v
        rocketVelocity = -rocketVelocity + 2 * celestial_bodies['Mars'].planet.v
        
        fuel -= mag(rocketVelocity) * 0.02
            
        rocket.pos += rocketVelocity
            
        d_dist_after_action = Agent.dist_mars_rocket()
        
        reward = -0.01 * d_dist_after_action
        
        if d_dist_after_action < d_dist_before_action:
            reward *= -1
            reward = 3 - reward
        
        if reward < -3:
            is_done = True
            reward = reward - (max_number_of_steps - total_steps) * 6
        
        reward -= (1 - fuel * 0.02)
        print(reward, fuel)
        
        total_steps += 1
        
        if Agent.calculate_dist_to_sun(rocket.pos) < celestial_bodies['Sun'].star.radius:
            reward -= 150
            is_done = True
        elif Agent.landed_on_earth():
            reward -= 20
            is_done = True
        elif Agent.landing():
            reward += 150
            is_done = True
        
        scene.capture('planets_positions{}'.format(frame_idx))
        sleep(1)
        next_state = imread('.\\Planets Positions - Images\\planets_positions{}.png'.format(frame_idx))
        next_state = next_state / 255
        remove('.\\Planets Positions - Images\\planets_positions{}.png'.format(frame_idx))
        return next_state, reward, is_done
            
        
    def play_step(self, net, epsilon, state):
        
        global total_reward
        
        action = 0
        done_reward = None
        
        if epsilon >= np.random.random():
            action = np.random.randint(8)
        else:
            action = np.argmax(net.predict(np.array([state])))
        
        new_state, reward, is_done = Agent.step(action)
        total_reward += reward
        
        exp = Experience(state, action, new_state, reward, is_done)
        self.exp_buffer.push(exp)
        self.state = new_state
        if is_done or total_steps == max_number_of_steps:
            done_reward = total_reward
            reset_env()
        return done_reward

<ul>
    <li><code>play_step()</code>: this function selects the action to take in the given situation. The action can be chosen randomly or the one with the highest value in the neural network. We save the results of each step and if we are ready we start again.</li>
    <li><code>step()</code>: the function performs the set action and receives the corresponding reward. The reward, the next_state of the environment and information on whether we have graduated are returned. Here's a little more information:</li>
    Rewards can vary greatly. Therefore, they are briefly described here:<br>
    1. <code>reward = -0.01 * Agent.dist_mars_rocket()</code> when the rocket moves away from Mars, it receives a negative reward.<br>
    2. <code>reward *= -1; reward = 3 - reward</code> when it's closer the reward is positive.<br>
    3. <code>reward = reward - (max_number_of_steps - total_steps) * 6</code> When our agent is more than 400 pixels from Mars (too far), he receives the following reward. It is because the later it moves away, the less the rocket is "punished", and the earlier, the more points are taken away from it. max_number_of_steps = 50 is the maximum number of moves per one attempt in the environment.<br>
    4. <code>reward -= 20</code> this is the case when the agent lands on the sun or on the ground of Earth again.<br>
    5. <code>reward -= 150</code> - the case when rocket hits the sun.<br>
    6. <code>reward += 150</code> when the rocket reaches its destination - Mars.
    <li><code>train()</code>: we choose a number of random examples (with the size of <code>batch_size</code>) from our memory - <code>ReplayMemory</code> and train the neural network to predict a future event based on the current one. Before training we take q-values from the prediction of actions from the neural network <code>net</code>. We replace the selected <code>action</code> with the <code>reward</code> if we are ready, otherwise we replace it with the sum of the <code>reward</code> and the further <code>action</code> we predict with <code>target_net</code>.</li>
</ul>
The other functions are clear - they compare the position of the rocket with the celestial bodies and help choose the <code>reward</code>.

Let's create other variables that we will need.

In [19]:
try:
    net, target_net = tf.keras.models.load_model('.\\weights.sav'), tf.keras.models.load_model('.\\weights.sav')
except:
    net, target_net = model, model
target_net.set_weights(net.get_weights())

buffer = ReplayMemory(replay_size)
agent = Agent(buffer)
epsilon = eps_start

optimizer = Adam(learning_rate)

total_rewards = []

frame = 0
frame_idx = 0
best_mean_reward = None

`net` and `target_net` are the two neural networks. `net` is learned using `target_net`, and then the weights of `target_net` are equated to `net`.<br>

`buffer` is an object of type `ReplayMemory` - the place where we will store information.<br>

`agent` is our agent (not the rocket, but the place where we store some important information and functions about the rocket).<br>

`total_rewards` is list with all of the rewards.<br>
`best_mean_reward` is the best mean reward.

`frame` is the number of steps that we are taking.<br>
`frame_idx` is total number of frames.<br>

In [10]:
def simulation():
    rate(8)

    dist_from_sun['Earth'] = (celestial_bodies['Earth'].planet.pos.x**2 + celestial_bodies['Earth'].planet.pos.y**2 + celestial_bodies['Earth'].planet.pos.z**2)**0.5
    dist_from_sun['Mars'] = (celestial_bodies['Mars'].planet.pos.x**2 + celestial_bodies['Mars'].planet.pos.y**2 + celestial_bodies['Mars'].planet.pos.z**2)**0.5

    radialVector = {
        'Earth': (celestial_bodies['Sun'].star.pos - celestial_bodies['Earth'].planet.pos) / dist_from_sun['Earth'],
        'Mars': (celestial_bodies['Sun'].star.pos - celestial_bodies['Mars'].planet.pos) / dist_from_sun['Mars']
    }
    
    Fgrav_earth_sun = grav_const * celestial_bodies['Earth'].mass * celestial_bodies['Sun'].mass * radialVector['Earth'] / dist_from_sun['Earth']**2
    Fgrav_mars_sun = 10 * grav_const * celestial_bodies['Mars'].mass * celestial_bodies['Sun'].mass * radialVector['Mars'] / dist_from_sun['Mars']**2
    Fgrav_earth_mars = grav_const * celestial_bodies['Earth'].mass * celestial_bodies['Mars'].mass * (celestial_bodies['Earth'].planet.pos - celestial_bodies['Mars'].planet.pos) / mag(celestial_bodies['Earth'].planet.pos - celestial_bodies['Mars'].planet.pos)**3
    
    celestial_bodies['Earth'].update_params(Fgrav_earth_sun + Fgrav_earth_mars)
    celestial_bodies['Mars'].update_params(Fgrav_mars_sun + Fgrav_earth_mars)
    
    celestial_bodies['Earth'].move()
    celestial_bodies['Mars'].move()

The `simulation()` function is a simple execution of the animation of the motion of celestial bodies. Newton's law of universal gravitaion is used. Earth and Mars move in an ellipse around the sun, which is Kepler's first law.

The following code starts the whole program. At each step we increase `frame` and `frame_idx`, as well as change the value of `epsilon`. We take one step of the simulation and take the image of the environment. We scale the image and perform a step. If we have enough data, we train the networks. At the end of the simulation we add the reward to `total_rewards` and if the mean reward from the last 10 performances is better than `best_mean_reward` we keep the weights of net. Training continues until the `mean_reward` of the last 10 times exceeds `MEAN_REWARD_BOUND`.

In [22]:
while True:
    frame += 1
    frame_idx += 1
    epsilon = max(epsilon * eps_decay, eps_min)
    
    simulation()
    
    scene.capture('planets_positions{}'.format(frame_idx))
    sleep(1)
    
    try:
        state = imread('.\\Planets Positions - Images\\planets_positions{}.png'.format(frame_idx))
    except:
        continue
    
    state = state / 255.0
    
    remove('.\\Planets Positions - Images\\planets_positions{}.png'.format(frame_idx))
    
    reward = agent.play_step(net, epsilon, state)
    
    if len(buffer.memory) > batch_size:
        agent.train(net, target_net)
    
    if reward is not None:
        target_net.set_weights(net.get_weights())
        total_rewards.append(reward)
        mean_reward = np.mean(total_rewards[-10:])
        print('%d: %d games, mean_reward %.3f, (epsilon %.2f)' % 
            (frame_idx, len(total_rewards), mean_reward, epsilon))
        
        if best_mean_reward is None or best_mean_reward < mean_reward:
            net.save('weights.sav')
            best_mean_reward = mean_reward
            print('Best mean reward updated %.3f' % (best_mean_reward))
                
        if mean_reward > MEAN_REWARD_BOUND:
            print('Solved in {} frames!'.format(frame_idx))
            break

-1.8285020618580177 94.46598691246702
-1.7947147998210176 94.40363985944244
-1.9946143166999926 93.93227088531343
-1.9707262250552162 93.87408049500168
-170.17892424381003 93.39176992069564
x_start_earth:  61
25: 3 games, mean_reward -172.244, (epsilon 0.24)
1.1847516613855178 99.90907498724806
x_start_earth:  -115
26: 4 games, mean_reward -133.887, (epsilon 0.24)
2.9878356308367824 99.85764009832641
2.9694191908952456 99.64876882608867
2.9353504879901875 99.43730266288921
2.989550431395176 99.16596548933444
2.9452789066239555 98.94912675530746
-0.27504713453851837 98.63536755715616
2.6637514690763187 98.41295259691695
-0.4101360942441896 98.18768741388693
2.5822439019714514 98.04662915256431
-0.5085488165349457 97.81556607745291
-0.6095189980188978 97.58156546463835
-0.7194971505992227 97.34460803482926
-0.8374431262303381 97.1046791268908
-0.9624559818688223 96.86176853298521
-1.0937678001407258 96.61587034216303
-1.2307297126452523 96.36698279254436
-1.372795292471224 96.11510813210

KeyboardInterrupt: 

In [26]:
net.save('weights.sav')

INFO:tensorflow:Assets written to: weights.sav\assets


# Thank you for your attention!